In [ ]:
# PART ONE: HTML PARSING REVIEW
# Lets grab some DoD selected acquisition reports!

import requests
from bs4 import BeautifulSoup

base_link = "http://www.dod.mil/pubs/foi/Reading_Room/Selected_Acquisition_Reports/"
response = requests.get(base_link)
assert response.status_code == 200
html_text = response.text
soup = BeautifulSoup(html_text,'html.parser')

all_a_tags = soup.find_all("a")

for a_tag in all_a_tags:
    file = open(a_tag.string,"wb")
    
    pdf_response = requests.get(base_link + a_tag['href']).content

    file.write(pdf_response)
    print("Downloaded " + a_tag.string)
    file.close()
    
print("Done!")

In [ ]:
# HTML PARSING from Archives.gov
# Example: Foundation of US Intelligence Documents

import requests
from bs4 import BeautifulSoup
import progressbar # If you don't have it yet, you'll need to do "pip install progressbar" in command prompt (without quotes)
# This is an optional add-on to produce a progress bar for a given for loop
bar = progressbar.ProgressBar()

base_link = 'https://history.state.gov/historicaldocuments/frus1945-50Intel/d'

# Why did I choose 1 to 72?
# You just wrap the normal for X in Y statement => for X in bar(Y):
# See the results when you run this cell
for index in bar(range(1,72)):
    
    # The following five lines should be familiar to everyone:
    hyperlink = base_link + str(index)
    response = requests.get(hyperlink)
    assert response.status_code == 200
    html_text = response.text
    soup = BeautifulSoup(html_text,"html.parser")
    # How is this finding the text I want?
    payload = {}
    payload['class'] = 'tei-div3'
    
    main_div_tag = soup.find("div", payload)
    core_text = main_div_tag.text
    
    # Now lets output it - a new variant on making a file
    file_name = "output_text" + str(index) + ".txt"
    fw = open(file_name,"w")
    fw.write(core_text)
    fw.close()
print("Done!")

In [ ]:
# PART TWO: API REVIEW
# OMDb Data (https://www.omdbapi.com/)

import requests
import pprint

base_link = 'http://www.omdbapi.com/'

payload = {}
payload['t'] = 'Boondock Saints'
payload['type'] = 'movie'

response = requests.get(base_link, params = payload)
assert response.status_code == 200

my_dict = response.json()
pprint.pprint(my_dict)

In [ ]:
my_dict['Ratings'][1]['Source']

In [ ]:
# What if we wanted to make a spreadsheet about the Big Bang Theory?
# I want Every Season, Every Episode, with the episode's name and IMDb rating!

# Let's do it together!
# Remember: There is a parameter 'Season' and a parameter 'Episode'
import requests
import pprint
import unicodecsv as csv

my_file = open("random_spreadsheet.csv","wb")
my_csv = csv.writer(my_file)

base_link = 'http://www.omdbapi.com/'
for index in range(1,11):
    payload = {}
    payload['t'] = 'The Big Bang Theory'
    payload['Season'] = index

    response = requests.get(base_link, params = payload)
    assert response.status_code == 200

    my_dict = response.json()
    #pprint.pprint(my_dict)

    for current_episode in my_dict['Episodes']:
        my_csv.writerow([current_episode['Title'],current_episode['imdbRating']])
print("Done!")
my_file.close()

In [ ]:
# PART THREE: SOMETHING NEW - NATURAL LANGUAGE PROCESSING IN PYTHON
# First, lets download some nltk add-ons
import nltk
print("Ready!")
nltk.download()
# When the new window loads, download ALL. Note that this will take a few hours.

In [ ]:
# Positive/Negative Sentiment Analysis
# Not good at handling more than a sentence or two at a time.
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

sentences = ["VADER is smart, handsome, and funny.", # positive sentence example
"VADER is smart, handsome, and funny!", # punctuation emphasis handled correctly (sentiment intensity adjusted)
"VADER is very smart, handsome, and funny.",  # booster words handled correctly (sentiment intensity adjusted)
"VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
"VADER is VERY SMART, handsome, and FUNNY!!!",# combination of signals - VADER appropriately adjusts intensity
"VADER is VERY SMART, really handsome, and INCREDIBLY FUNNY!!!",# booster words & punctuation make this close to ceiling for score
"The book was good.",         # positive sentence
"The book was kind of good.", # qualified positive sentence is handled correctly (intensity adjusted)
"The plot was good, but the characters are uncompelling and the dialog is not great.", # mixed negation sentence
"A really bad, horrible book.",       # negative sentence with booster words
"At least it isn't a horrible book.", # negated negative sentence with contraction
":) and :D",     # emoticons handled
"",              # an empty string is correctly handled
"Today sux",     #  negative slang handled
"Today sux!",    #  negative slang with punctuation emphasis handled
"Today SUX!",    #  negative slang with capitalization emphasis
"Today kinda sux! But I'll get by, lol" # mixed sentiment example with slang and constrastive conjunction "but"
]

tricky_sentences = [
"Most automated sentiment analysis tools are shit.",
"VADER sentiment analysis is the shit.",
"Sentiment analysis has never been good.",
"Sentiment analysis with VADER has never been this good.",
"Warren Beatty has never been so entertaining.",
"I won't say that the movie is astounding and I wouldn't claim that \
the movie is too banal either.",
"I like to hate Michael Bay films, but I couldn't fault this one",
"It's one thing to watch an Uwe Boll film, but another thing entirely \
to pay for it",
"The movie was too good",
"This movie was actually neither that funny, nor super witty.",
"This movie doesn't care about cleverness, wit or any other kind of \
intelligent humor.",
"Those who find ugly meanings in beautiful things are corrupt without \
being charming.",
"There are slow and repetitive parts, BUT it has just enough spice to \
keep it interesting.",
"The script is not fantastic, but the acting is decent and the cinematography \
is EXCELLENT!",
"Roger Dodger is one of the most compelling variations on this theme.",
"Roger Dodger is one of the least compelling variations on this theme.",
"Roger Dodger is at least compelling as a variation on the theme.",
"they fall in love with the product",
"but then it breaks",
"usually around the time the 90 day warranty expires",
"However, Mr. Carter solemnly argues, his client carried out the kidnapping \
under orders and in the ''least offensive way possible.''",
]

sentences.extend(tricky_sentences)

sid = SentimentIntensityAnalyzer()
for sentence in sentences:
    print(sentence)
    ss = sid.polarity_scores(sentence)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]), end='')
    print()

In [ ]:
# Next, lets get an API Key from Indico.io
# Visit their website at indico.io, and make a free account.
# When they give you an API key (check out indico.io/dashboard when you have an account)
# Replace YOUR-API-KEY with the API key they give you.
# Political Sentiment Analysis

from indicoio import political, config
config.api_key = "YOUR-API-KEY"
print(political("Those who surrender freedom for security will not have, nor do they deserve, either one."))
print(political("War solves no problems. Only peace can achieve the future we all deserve."))
print(political("Fracking is evil, it destroys the environment. We need to utilize solar and wind energy."))

In [ ]:
# Emotional Sentiment Analysis
# Surprisingly accurate!
from indicoio import emotion
print(emotion("The evil Lord Voldemort is going to destroy Hogwarts."))
print(emotion("I can't believe that John Doe won the election... I wasn't expecting it at all!"))
print(emotion("I just failed the test, I'm going to rage at the world"))

In [ ]:
# Text Tagging by 110 common writing categories
# Sorting text into pre-defined categories, with likelihood of fit probabilities
from indicoio import text_tags, keywords
print('Text Tags:')
print(text_tags("Facebook blog posts about Android tech make better journalism than most news outlets.", top_n=5))

In [ ]:
# Keyword determination in text
# Figure out the most IMPORTANT words in a given sentence or paragraph
print('Keywords:')
print(keywords("Facebook blog posts about Android tech make better journalism than most news outlets.", top_n=5))